In [13]:
import requests
import json
import pandas as pd
import geopandas as gpd
import contextily as ctx
import geoplot
import matplotlib.pyplot as plt
import folium
import pandas as pd

In [14]:
api_url = "https://data.lillemetropole.fr/data/ogcapi/collections/vlille_temps_reel/items?f=geojson&limit=-1"
api_call = requests.get(api_url)


In [15]:
api_data = api_call.text
api_data = json.loads(api_data)
df = [feature for feature in api_data['features']]
df = pd.json_normalize(df)


In [16]:
#Position [latitude, longitude] sur laquelle est centrée la carte
location = [50.641926, 3.075992]

#Niveau de zoom initial : 
#3-4 pour un continent, 5-6 pour un pays, 11-12 pour une ville
zoom = 12

#Style de la carte
tiles = 'cartodbpositron'

Carte = folium.Map(location = location,
                   zoom_start = zoom,
                   tiles = tiles)
#Création du marqueur
for i in range(len(df)):
    point1 = [df['geometry.coordinates'][i][1], df['geometry.coordinates'][i][0]]
    text = f"Nom de l'arrêt {df['properties.nom'][i]} <br> Nombre de vélos disponibles {df['properties.nb_velos_dispo'][i]} <br> Nombre de places disponibles {df['properties.nb_places_dispo'][i]}"
    marqueur = folium.Marker(location = point1,
                        popup = text)
    #Ajout à la carte
    marqueur.add_to(Carte)

display(Carte)


In [19]:
import plotly.express as px
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from geodatasets import get_path

fig = px.scatter_mapbox(df,
                        lat="properties.y",
                        lon="properties.x", 
                        hover_data = {"properties.nb_places_dispo" : True,
                            "properties.nb_velos_dispo" : True},
                        color="properties.nom",
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max=15, 
                        zoom=10)
fig.update_layout(mapbox_style="open-street-map")

fig.show()